In [1]:
import hublib
import hublib.use
%use octave-4.2.0
import oct2py
%load_ext oct2py.ipython
import pandas as pd
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib
import matplotlib.pyplot as plt
import scipy
import numpy as np
from decimal import Decimal
%octave pkg load optim
import unicodedata
from unicodedata import lookup as GL
import sympy as sy
import time

In [2]:
from hublib.ui import FileUpload

def mycb(name,val):
    #print("CB %s (length = %s bytes)" % (name, len(val)))
    global fnm
    fnm=name

f = FileUpload("Please upload Raman spectra data file (CSV)", 
               "Raman data files should be uploaded as 2 column CSV files",cb=mycb,maxsize=10000000)
f

<IPython.core.display.Javascript object>

Box(children=(HTML(value='<p data-toggle="popover" title="Raman data files should be uploaded as 2 column CSV files">Please upload Raman spectra data file (CSV)</p>', layout=Layout(flex='2 1 auto')), FileWidget(maxsize=10000000)), layout=Layout(border='solid 1px lightgray', display='flex', flex_flow='row', justify_content='space-between', width='auto'))

In [3]:
def errprint(code):
    errfile=pd.read_csv('errfile.txt',sep='\t',header=None)
    with errout:
        clear_output()
        print(errfile[0][code])
        errout

In [4]:
def case_lookup(index):
    casefile=pd.read_csv('Case_List.txt',sep='\t',header=None)
    c=casefile[0][index]
    return c

In [17]:
fit_but = widgets.Button(description='Do Fitting')
    
def fit_but_cb(change):
    
    with errout:
        clear_output()
    with diffsplot:
        clear_output()
    with datplot:
        clear_output()
    with plist:
        clear_output()
        
    f.save('tmpfile')
    if fnm[-3:]=='txt':
        sp='\t'
    elif fnm[-3:]=='csv':
        sp=','
    else:
        errprint(0)
        return
    
    with plist:
        clear_output()
        print('Reading daata file...')
    data = pd.read_csv('tmpfile',sep=sp,header=None)
    with plist:
        clear_output()
        print('Data file read')
    
    n=int(data.size/len(data)) #n determines the size of the data file

    ##Single Spectra Data File, n=2    
    if n==2:
        with plist:
            clear_output()
            print('Fitting single spectra data.')
        I,W=Spectra(data)
    
        PG,PGp,If,PD,IDfit,Q,diffs,mdi,md,mf=Fit(I,W)

        dtplot(I,W,If,IDfit,mf,Q,PG,PGp,PD)

        with diffsplot:
            clear_output()
            plt.plot(diffs,'kv')
            plt.plot(mdi,md,'gv')
            plt.annotate((round(Decimal(md),2)),xy=(mdi,1.2*md))
            plt.xticks(range(6),('1','2','3','4','5','Graphite'))
            plt.xlabel('# Layers')
            plt.ylabel('$\Delta$ [%]')
            plt.show()

        with plist:
            clear_output()
            G=GL('GREEK CAPITAL LETTER GAMMA')
            o=GL('GREEK SMALL LETTER OMEGA')
            print('G Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
                  'G\' Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
                  'D Fitting Parameters:\n\tA=%1.2f\n\t%s=%1.2f\n\t%s=%1.2f\n'
                  'Quality=%1.2f (Ratio of D to G)\n'
                  'Best Case Match: %s'
                  %(PG[0],G,PG[1],o,PG[2],PGp[0],G,PGp[1],o,PGp[2],PD[0],G,PD[1],o,PD[2],Q,case_lookup(mdi)))
    #Map files will be much larger than 2 points and need separate handling
    elif n > 2:
        Map(data)
    else:
        errprint(0)
        return
    
fit_but.on_click(fit_but_cb)
fit_but

Button(description='Do Fitting', style=ButtonStyle())

In [16]:
def Map(data):
    global x,y
    t1=time.clock()
    T1=t1
    W=data[:][0:1]
    W=np.array(W)
    W=W[~np.isnan(W)]
    
    x=data[0]
    x=np.array(x)
    x=x[~np.isnan(x)]
    xu=np.unique(x)
    
    y=data[1]
    y=np.array(y)
    y=y[~np.isnan(y)]
    yu=np.unique(y)
    
    n=yu.size*xu.size
    
    xi = np.linspace(min(x), max(x))
    yi = np.linspace(min(y), max(y))
    X, Y = np.meshgrid(xi, yi)
    
    global IG,IGp,ID,wG,wGp,wD,GG,GGp,GD
    
    PG=np.zeros((n,3));PGp=np.zeros((n,3));PD=np.zeros((n,3));
    diffs=np.zeros((n,5));
    Q=np.zeros(n);mdi=np.zeros(n);md=np.zeros(n);
    I=np.zeros((n,1024));If=np.zeros((n,1024));mf=np.zeros((n,1024));IDfit=np.zeros((n,1024))
    
    wG=np.zeros(n)
    wGp=np.zeros(n)
    wD=np.zeros(n)
    IG=np.zeros(n)
    IGp=np.zeros(n)
    ID=np.zeros(n)
    GG=np.zeros(n)
    GGp=np.zeros(n)
    GD=np.zeros(n)
    
    for k in range(n):
        t2=time.clock()
        dt=(t2-t1)
        t1=t2
        pdone=100*(k+1)/n
        t_e=(n-k)*dt/60
        if t_e<60:
            ts='sec'
            t_est=t_e*60
        else:
            ts='min'
            t_est=t_e
        #print(t_est)
        with plist:
            clear_output()
            print('Fitting map data. This may take some time...\n%1.2f%% Done\nEstimated time left [%s]: %1.2f'%(pdone,ts,t_est))
        
        I_raw=np.array(data)[k+1][2:1026]
        i=((I_raw-np.min(I_raw))/np.max(I_raw-np.min(I_raw)))
        pG,pGp,i_f,pD,iDfit,q,Diffs,Mdi,Md,Mf=Fit(i,W)
        
        I[k]=np.transpose(i)
        If[k]=np.transpose(i_f)
        mf=np.transpose(Mf)
        #diffs[k]=np.transpose(Diffs)
        Q[k]=q
        mdi[k]=Mdi
        md[k]=Md
        
        for j in range(3):
            PG[k][j]=pG[j]
            PGp[k][j]=pGp[j]
            PD[k][j]=pD[j]
        
        IG[k]=pG[0]
        IGp[k]=pGp[0]
        ID[k]=pD[0]
        
        GG[k]=pG[1]
        GGp[k]=pGp[1]
        GD[k]=pD[1]
        
        wG[k]=pG[2]
        wGp[k]=pGp[2]
        wD[k]=pD[2]
        #dtplot(i,W,i_f,iDfit,Mf,q,pG,pGp,pD)

    Mplot(x,y,wG)
    with diffsplot:
        clear_output()
        plt.hist(wG,bins='auto')
        plt.show()
    T2=time.clock()
    with plist:
        clear_output()
        print('Fitting Finished\nActual time [min]: %1.2f'%((T2-T1)/60))

In [20]:
def Mplot(x,y,z):
    xi = np.linspace(min(x), max(x))
    yi = np.linspace(min(y), max(y))
    X, Y = np.meshgrid(xi, yi)
    
    with datplot:
        clear_output()
        Z=matplotlib.mlab.griddata(x, y, z, xi, yi, interp='linear') 
        plt.contourf(X,Y,Z)
        plt.set_cmap('inferno')
        plt.xlabel('x [mm]')
        plt.ylabel('y [mm]')
        plt.show()

In [8]:
def Spectra(data):
    srow=0;
    if type(data[0][0])==str:
        srow=1
    
    W=data[0][srow:len(data)]
    W=np.array(W);W=W.astype(float)
    I_raw=data[1][srow:len(data)]
    I_raw=np.array(I_raw);I_raw=I_raw.astype(float)
    I=((I_raw-np.min(I_raw))/np.max(I_raw-np.min(I_raw)));
    return I,W

In [9]:
def Fit(I,W):
    pG=[np.max(I), 5, 1581.6] #a w b
    pGp=[np.max(I), 18, 2675]
    PG=%octave -i pG,W,I lsqcurvefit(@Single_Lorentz,pG,W,I,[0,0,1400],[max(I),50,2000]);
    PGp=%octave -i pGp,W,I lsqcurvefit(@Single_Lorentz,pGp,W,I,[0,0,2000],[max(I),100,3000]);
    PG[1]=np.absolute(PG[1]);PGp[1]=np.absolute(PGp[1]); #FWHM sometimes returns - bc always squared
    IGfit=%octave -i PG,W Single_Lorentz(PG,W);
    IGpfit=%octave -i PGp,W Single_Lorentz(PGp,W);
    IGfit=IGfit[0];IGpfit=IGpfit[0];
    If=IGfit+IGpfit;
    
    
    pD=[np.max(I),5,1300]
    PD=%octave -i pD,W,I lsqcurvefit(@Single_Lorentz,pD,W,I,[0,0,1200],[max(I),50,1400]);
    PD[1]=np.absolute(PD[1]);
    IDfit=%octave -i PD,W Single_Lorentz(PD,W);
    IDfit=IDfit[0]
    Q=1-(PD[0]/PG[0])
    Q=Decimal(Q[0])
    
    Cdat=np.load('Cfits.npy')

    diffs_lin=[];diffs_Gp=[];
    diffs=[];
    for d in range(6):
        
        LG=Cdat[d][0];
        LGp=Cdat[d][1];
        LGfit=%octave -i LG,W Single_Lorentz(LG,W); 
        LGpfit=%octave -i LGp,W Single_Lorentz(LGp,W);
        Lf=(LGfit+LGpfit)
 
        dfGp=np.average(np.absolute(100*(PGp-LGp)/LGp))
        dfG=np.average(np.absolute(100*(PG-LG)/LG))
        df=np.average([dfGp,dfG])
        diffs.append(df)
  
    md=np.min(diffs)
    mdi=np.argmin(diffs)

    mG=Cdat[mdi][0];mGp=Cdat[mdi][1];
    mGfit=%octave -i mG,W Single_Lorentz(mG,W);
    mGpfit=%octave -i mGp,W Single_Lorentz(mGp,W);
    mf=mGfit+mGpfit;mf=mf[0]
    return PG,PGp,If,PD,IDfit,Q,diffs,mdi,md,mf

In [10]:
def dtplot(I,W,If,IDfit,mf,Q,PG,PGp,PD):
        with datplot:
            clear_output()
            plt.plot(W,I,'b',W,If+IDfit,'r',W,mf,'g')
            plt.xlabel('$\omega$ $[cm^{-1}]$')
            plt.ylabel('$I_{norm}$ [arb]')
            plt.legend(labels=['Raw','Fit','Test'])
            plt.annotate('Q=%1.2f' %round(Q,2) ,xy=(np.min(W),0.98))
            plt.annotate('D',xy=(0.85*PD[2],1.1*PD[0]))
            plt.annotate('G',xy=(0.9*PG[2],0.95*PG[0]))
            plt.annotate('G\'',xy=(0.94*PGp[2],0.95*PGp[0]))
            plt.show()

In [11]:
datplot=widgets.Output();datplot

Output()

In [12]:
diffsplot=widgets.Output();diffsplot

Output()

In [13]:
errout=widgets.Output();errout

Output()

In [14]:
plist=widgets.Output();plist

Output()

In [15]:
o=GL('GREEK SMALL LETTER OMEGA')
G=GL('GREEK CAPITAL LETTER GAMMA')

def param_change(change):
    if param.value==1:
        z=IG
    elif param.value==2:
        z=wG
    elif param.value==3:
        z=GG
    elif param.value==4:
        z=IGp
    elif param.value==5:
        z=wGp
    elif param.value==6:
        z=GGp
    elif param.value==7:
        z=ID
    elif param.value==8:
        z=wD
    elif param.value==9:
        z=GD
    else:
        z=wG
    
    Mplot(x,y,z)

param=widgets.Dropdown(description='Parameter')
param.options=options={'Select': '', 'I_G': 1, (o+'_G'): 2, (G+'_G'): 3, 'I_G\'': 4, (o+'_G\''): 5, (G+'_G\''): 6, 'I_D': 7, (o+'_D'): 8, (G+'_D'): 9}
param.observe(param_change)
param

Dropdown(description='Parameter', options={'Select': '', 'I_G': 1, 'ω_G': 2, 'Γ_G': 3, "I_G'": 4, "ω_G'": 5, "Γ_G'": 6, 'I_D': 7, 'ω_D': 8, 'Γ_D': 9}, value='')